**Prepare the dataset**

In [5]:
import os
import random
import shutil

# Define paths
images_folder = "yolo_data/images"
labels_folder = "yolo_data/labels"
train_images_folder = "yolo_data_split/images/train"
val_images_folder = "yolo_data_split/images/val"
train_labels_folder = "yolo_data_split/labels/train"
val_labels_folder = "yolo_data_split/labels/val"

# Create directories for train and val splits
os.makedirs(train_images_folder, exist_ok=True)
os.makedirs(val_images_folder, exist_ok=True)
os.makedirs(train_labels_folder, exist_ok=True)
os.makedirs(val_labels_folder, exist_ok=True)

# Get list of image files and corresponding labels
image_files = os.listdir(images_folder)
image_files = [f for f in image_files if f.endswith(".jpg") or f.endswith(".jpeg") or f.endswith(".png")]  # Filter out only the image files

# Shuffle the dataset
random.seed(42)  # For reproducibility
random.shuffle(image_files)

# Define the split ratio
train_ratio = 0.8  # 80% for training, 20% for validation
split_index = int(len(image_files) * train_ratio)

# Split the data
train_images = image_files[:split_index]
val_images = image_files[split_index:]

# Move images and labels to corresponding folders
def move_files(image_list, destination_image_folder, destination_label_folder):
    for image_file in image_list:
        # Move the image file
        image_path = os.path.join(images_folder, image_file)
        shutil.move(image_path, os.path.join(destination_image_folder, image_file))
        
        # Move the corresponding label file
        label_file = image_file.replace(".jpg", ".txt").replace(".jpeg", ".txt").replace(".png", ".txt")
        label_path = os.path.join(labels_folder, label_file)
        shutil.move(label_path, os.path.join(destination_label_folder, label_file))

# Move training data
move_files(train_images, train_images_folder, train_labels_folder)

# Move validation data
move_files(val_images, val_images_folder, val_labels_folder)

print(f"Training set: {len(train_images)} images")
print(f"Validation set: {len(val_images)} images")


Training set: 5 images
Validation set: 2 images


In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

# Use the model
results = model.train(data="config.yaml", epochs=3)  # train the model
results = model.val()  # evaluate model performance on the validation set
results = model("https://ultralytics.com/images/bus.jpg")  # predict on an image
success = model.export(format="onnx")  # export the model to ONNX format